In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import boto3
import json
import matplotlib.pyplot as plt
import requests
import pathlib
import utils.configs as configs
import utils.strava as strava

plt.style.use('seaborn-v0_8-pastel')

In [3]:
s3 = boto3.resource('s3')
bucket_name = "strava-raw"

resp = strava.auth().json()
access_token = resp['access_token']

[WDM] - Downloading: 100%|██████████| 6.58M/6.58M [00:00<00:00, 18.4MB/s]


In [25]:
activity_ids = []
for obj in s3.Bucket(bucket_name).objects.all():
    if pathlib.Path(obj.key).stem.isnumeric():
        body_bytes = obj.get()['Body'].read()
        activity_ids.append(json.loads(body_bytes)['id'])

# working with a subset of activities
activity_ids = activity_ids[:10]

In [28]:
tables = ["zones", "laps", "streams"]

for table in tables:
    data = strava.batch_get_request(table, activity_ids, access_token) 
    s3.Object(bucket_name, f'{table}.json').put(Body=json.dumps(data))


zones: executing request 10 of 10
laps: executing request 10 of 10
streams: executing request 10 of 10
